In [1]:
import pandas as pd
from pandarallel import pandarallel
from typing import defaultdict

In [2]:
data_path = '..\\data\\'

In [3]:
clients_cleaned = pd.read_feather(data_path + '\\interim\\clnts.frt')

In [4]:
def columns_report(data: pd.DataFrame) -> None:

    print('Rows:', data.shape[0], end='\n'*2)

    stats = defaultdict(list)
    for col in data.columns:
        stats['Column'].append(col)
        stats['Unique'].append(data[col].nunique())
        stats['Duplicates'].append(data[col].duplicated().sum())
        stats['Missing'].append(data[col].isna().sum())
        stats['Missing%'].append(data[col].isna().mean() * 100)
        stats['HitRate%'].append((1 - data[col].isna().mean()) * 100)
    stats = pd.DataFrame(stats)

    print(stats.to_string(index=False))

### Соотношение среднедушевых денежных доходов населения с величиной прожиточного минимума (процент)

In [5]:
avgpercapinc_subsistmin_ratio = pd.read_excel(data_path + 'external\\соотношение_среднедушевых_доходов_с_прожиточным_минимумом_по_субъектам.xls',
                                              sheet_name='long_fmt')
print(avgpercapinc_subsistmin_ratio.shape)

(5357, 4)


In [6]:
avgpercapinc_subsistmin_ratio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5357 entries, 0 to 5356
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Субъект   5357 non-null   object 
 1   Год       5357 non-null   int64  
 2   Квартал   5357 non-null   object 
 3   Значение  5357 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 167.5+ KB


In [7]:
columns_report(avgpercapinc_subsistmin_ratio)

Rows: 5357

  Column  Unique  Duplicates  Missing  Missing%  HitRate%
 Субъект     102        5255        0       0.0     100.0
     Год      26        5331        0       0.0     100.0
 Квартал       4        5353        0       0.0     100.0
Значение    2393        2964        0       0.0     100.0


In [8]:
subjects = ['край', 'область', 'республика', 'москва', 'санкт-петербург', 'российская федерация']
stop_words = subjects[:3]


def rm_stop_words(s: str, stop_words=stop_words):

    return ' '.join(w for w in s.split() if w not in stop_words)

In [9]:
pandarallel.initialize(progress_bar=False, nb_workers=12)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [10]:
avgpercapinc_subsistmin_ratio['Субъект'] = avgpercapinc_subsistmin_ratio.Субъект\
    .str.strip()\
    .str.lower()\
    .str.replace(r'\s*\(.*\)', '', regex=True)\
    .str.replace(r' - .*', '', regex=True)
avgpercapinc_subsistmin_ratio = avgpercapinc_subsistmin_ratio.loc[
    avgpercapinc_subsistmin_ratio.Субъект.str.contains('|'.join(subjects))].reset_index(drop=True)
avgpercapinc_subsistmin_ratio['Субъект'] = avgpercapinc_subsistmin_ratio.Субъект.parallel_apply(
    rm_stop_words)\
    .str.strip()

clients_cleaned['geo'] = clients_cleaned.geo\
    .str.strip()\
    .str.lower()\
    .str.replace(r'\s*\(.*\)', '', regex=True)\
    .parallel_apply(rm_stop_words)\
    .str.strip()

In [11]:
avgpercapinc_subsistmin_ratio.Субъект.unique()

array(['российская федерация', 'белгородская', 'брянская', 'владимирская',
       'воронежская', 'ивановская', 'калужская', 'костромская', 'курская',
       'липецкая', 'московская', 'орловская', 'рязанская', 'смоленская',
       'тамбовская', 'тверская', 'тульская', 'ярославская', 'москва',
       'карелия', 'коми', 'архангельская', 'вологодская',
       'калининградская', 'ленинградская', 'мурманская', 'новгородская',
       'псковская', 'санкт-петербург', 'адыгея', 'калмыкия', 'крым',
       'краснодарский', 'астраханская', 'волгоградская', 'ростовская',
       'дагестан', 'ингушетия', 'кабардино-балкарская',
       'карачаево-черкесская', 'северная осетия-алания', 'чеченская',
       'ставропольский', 'башкортостан', 'марий эл', 'мордовия',
       'татарстан', 'удмуртская', 'чувашская', 'пермский', 'кировская',
       'нижегородская', 'оренбургская', 'пензенская', 'самарская',
       'саратовская', 'ульяновская', 'курганская', 'свердловская',
       'тюменская', 'челябинская', 'алт

In [12]:
clients_cleaned.geo.unique()

array(['ленинградская', 'липецкая', 'вологодская', 'новгородская',
       'санкт-петербург', 'тюменская', 'свердловская', 'ставропольский',
       'нижегородская', 'оренбургская', 'курская', 'московская',
       'карелия', 'архангельская', 'самарская', 'рязанская', 'москва',
       'орловская', 'белгородская', 'калининградская', 'дагестан',
       'ростовская', 'волгоградская', 'башкортостан', 'хабаровский',
       'коми', 'адыгея', 'краснодарский', 'томская', 'удмуртская',
       'красноярский', 'алтайский', 'пермский', 'иркутская',
       'владимирская', 'смоленская', 'челябинская', 'новосибирская',
       'саратовская', 'воронежская', 'татарстан', 'чеченская',
       'пензенская', 'ульяновская', 'тамбовская', 'псковская',
       'сахалинская', 'брянская', 'мурманская', 'хакасия', 'ярославская',
       'приморский', 'курганская', 'кировская', 'мордовия', 'омская',
       'ивановская', 'астраханская', 'тульская', 'бурятия', 'чувашская',
       'кемеровская', 'калужская', 'амурская', '